## Data Warehousing Lab 3
Abu Abdullah Dhrubo<br>Ahmad Abdilrahim<br>Nahom Gebremeskel<br>Chinonso Williams Ubani

In [1]:
# Installing the psycopg2 library
# pip install psycopg2-binary

In [2]:
#Importing the libraries
import psycopg2
from configparser import ConfigParser
import psycopg2.extras as extras
from datetime import datetime
import pandas as pd

Before connecting we need to configure the database name and password in the config file manually. 

### Testing the connection to PostgreSQL

In [3]:
def config(filename='database.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db


""" Connect to the PostgreSQL database server """
conn = None
try:
    # read connection parameters
    params = config()

    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(**params)

    # create a cursor
    cur = conn.cursor()

    # execute a statement
    print('PostgreSQL database version:')
    cur.execute('SELECT version()')

    # display the PostgreSQL database server version
    db_version = cur.fetchone()
    print(db_version)

    # close the communication with the PostgreSQL
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(f"Error: {error}")
finally:
    if conn is not None:
        conn.close()
        print('Database connection closed.')

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 15.1, compiled by Visual C++ build 1914, 64-bit',)
Database connection closed.


### Creating the Tables to populate data from the working directory

In [4]:
""" create tables in the PostgreSQL database"""
commands = (
    """
  CREATE TABLE product (
  product_key integer NOT NULL,
  description varchar(200) NOT NULL,
  full_description varchar(200) NOT NULL,
  SKU_number bigint NOT NULL,
  package_size varchar(200) NOT NULL,
  brand varchar(200) NOT NULL,
  subcategory varchar(200) NOT NULL,
  category varchar(200) NOT NULL,
  department varchar(200) NOT NULL,
  package_type varchar(200) NOT NULL,
  diet_type varchar(200) NOT NULL,
  weight varchar(200) NOT NULL,
  weight_unit_of_measure varchar(200) NOT NULL,
  units_per_retail_case integer NOT NULL,
  units_per_shipping_case integer NOT NULL,
  cases_per_pallet integer NOT NULL,
  shelf_width_cm integer NOT NULL,
  shelf_height_cm integer NOT NULL,
  shelf_depth_cm integer NOT NULL,
  CONSTRAINT product_key_pk PRIMARY KEY (product_key))
    """,
    """
CREATE TABLE promotion (
promotion_key integer NOT NULL,
promotion_name varchar(200) NOT NULL,
price_reduction_type varchar(200) NOT NULL,
ad_type varchar(200) NOT NULL,
display_type varchar(200) NOT  NULL,
coupon_type varchar(200) NOT NULL,
ad_media_type varchar(200) NOT NULL,
display_provider varchar(200) NOT NULL,
promo_cost integer NOT NULL,
promo_begin_date timestamp NOT NULL,
promo_end_date timestamp NOT NULL ,
CONSTRAINT promotion_key_pk PRIMARY KEY (promotion_key))
    """,
    """

CREATE TABLE store (
store_key integer NOT NULL,
name varchar(200) NOT NULL,
store_number integer NOT NULL,
store_street_address varchar(200) NOT NULL,
city varchar(200) NOT NULL,
store_county varchar(200) NOT NULL,
store_state	varchar(200) NOT NULL,
store_zip varchar(200) NOT NULL,
sales_district varchar(200) NOT NULL,
sales_region varchar(200) NOT NULL,
store_manager varchar(200) NOT NULL,
store_phone varchar(200) NOT NULL,
store_FAX varchar(200) NOT NULL,
floor_plan_type varchar(200)	NOT NULL,
photo_processing_type varchar(200) NOT NULL,
finance_services_type varchar(200) NOT NULL,
first_opened_date timestamp NOT NULL,
last_remodel_date timestamp NOT NULL,
store_sqft integer NOT NULL,
grocery_sqft integer NOT NULL,
frozen_sqft	integer NOT NULL,
meat_sqft integer NOT NULL,
CONSTRAINT store_key_pk PRIMARY KEY (store_key))
    """,
    """
    CREATE TABLE time (
time_key integer NOT NULL,
date timestamp NOT NULL,
day_of_week varchar(200) NOT NULL,
day_number_in_month integer	NOT NULL,
day_number_overall integer NOT NULL,
week_number_in_year integer	NOT NULL,
week_number_overall integer	NOT NULL,
Month integer NOT NULL,
quarter integer NOT NULL,
fiscal_period varchar(200) NOT NULL,
year integer NOT NULL,
holiday_flag varchar(200) NOT NULL,
CONSTRAINT time_key_pk PRIMARY KEY (time_key));
    """,
    """
CREATE TABLE sales_facts (
product_key integer NOT NULL,
promotion_key integer NOT NULL,
store_key integer NOT NULL,
time_key integer NOT NULL,
dollar_sales integer NOT NULL,
unit_sales integer NOT NULL,
dollar_cost	integer NOT NULL,
customer_count integer NOT NULL,
CONSTRAINT fk_product_key FOREIGN KEY(product_key) REFERENCES product(product_key),
CONSTRAINT fk_promotion_key FOREIGN KEY(promotion_key) REFERENCES promotion(promotion_key),
CONSTRAINT fk_store_key FOREIGN KEY(store_key) REFERENCES store(store_key),
CONSTRAINT fk_time_key FOREIGN KEY(time_key) REFERENCES time(time_key))
    """
)
conn = None
try:
    # read the connection parameters
    params = config()
    # connect to the PostgreSQL server
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    # create table one by one
    for command in commands:
        cur.execute(command)
    # close communication with the PostgreSQL database server
    cur.close()
    # commit the changes
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

### Creating a function to populate the tables from the csv files.

In [5]:
def execute_values(conn, df, table):
    tuples = [tuple(x) for x in df.to_numpy()]

    cols = ','.join(list(df.columns))
    # SQL query to execute
    q = "SET datestyle = dmy;"
    query = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.execute(q)
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("The dataframe is inserted")
    cursor.close()

### Reading the data from the CSVs to the pandas dataframes

we are reading the data in pandas to manupulate and clean the date to easily populate in the PostgreSQL

In [6]:
products = pd.read_csv('product.csv')
promotions = pd.read_csv('promotion.csv')
stores = pd.read_csv('store.csv')
times = pd.read_csv('time.csv')
sales_facts = pd.read_csv('salesFact.csv')

for  index, row in times.iterrows():
    fix_month = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + row['Month'] - 2).timetuple().tm_mon
    times.at[index, 'Month'] = fix_month

here we are writing the data to the database

In [7]:
conn = psycopg2.connect(**params)
execute_values(conn, products, 'product')
execute_values(conn, promotions, 'promotion')
execute_values(conn, stores, 'store')
execute_values(conn, times, 'time')
execute_values(conn, sales_facts, 'sales_facts')

The dataframe is inserted
The dataframe is inserted
The dataframe is inserted
The dataframe is inserted
The dataframe is inserted


### We are answering the questions using SQL queries

• How many sales did the company have?  

• How many products did we sale in a store last month?

• Which one of our stores have the highest amount of sells?

• Which products are the most lucrative?

• What was the most lucrative day, month, or year?

In [8]:
""" Connect to the PostgreSQL database server """
conn = None
try:
    # read connection parameters
    params = config()

    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(**params)

    # create a cursor
    cur = conn.cursor()

    print('Question 1: How many sales did the company have?')
    sql1 = '''SELECT SUM(unit_sales) AS unit_sales
FROM sales_facts;'''
    cur.execute(sql1)
    for i in cur.fetchall():
        print(i)
    print('Question 2: How many products did we sale in a store last month?')
    sql1 = '''
SELECT SUM(unit_sales) AS unit_sales
FROM sales_facts
WHERE time_key in (SELECT time_key FROM time WHERE date BETWEEN '1995-12-01' AND '1995-12-31')
AND store_key in (SELECT store_key FROM store WHERE store_number = '10');
'''
    cur.execute(sql1)
    for i in cur.fetchall():
        print(i)

    print('Question 3: Which one of our stores have the highest amount of sells?')
    sql1 = '''
SELECT store_number, total_sales
FROM (
  SELECT s.store_number, SUM(sf.dollar_sales) as total_sales
  FROM sales_facts sf
  JOIN store s ON s.store_key = sf.store_key
  GROUP BY s.store_key
  ORDER BY total_sales DESC
  LIMIT 1
) t;
'''
    cur.execute(sql1)
    for i in cur.fetchall():
        print(i)
    print('Question 4: Which products are the most lucrative?')
    sql1 = '''SELECT product.description as product_name , SUM(sales_facts.dollar_sales) - SUM(sales_facts.dollar_cost) as profit
FROM sales_facts 
JOIN product ON product.product_key = sales_facts.product_key
GROUP BY product.description
ORDER BY profit desc;
'''
    cur.execute(sql1)
    for i in cur.fetchall():
        print(i)
    print('Question 5 : What was the most lucrative year?')
    sql1 = '''SELECT time.year, SUM(sales_facts.dollar_sales) - SUM(sales_facts.dollar_cost) as profit
FROM sales_facts 
JOIN time ON time.time_key = sales_facts.time_key
GROUP BY time.year
ORDER BY profit desc;'''
    cur.execute(sql1)
    for i in cur.fetchall():
        print(i)
    print('Question 5: What was the most lucrative day of the week?')
    sql1 = '''SELECT time.day_of_week, SUM(sales_facts.dollar_sales) - SUM(sales_facts.dollar_cost) as profit
FROM sales_facts 
JOIN time ON time.time_key = sales_facts.time_key
GROUP BY time.day_of_week
ORDER BY profit desc;'''
    cur.execute(sql1)
    for i in cur.fetchall():
        print(i)
    print('Question 5 : What was the most lucrative month?')
    sql1 = '''SELECT time.month, SUM(sales_facts.dollar_sales) - SUM(sales_facts.dollar_cost) as profit
FROM sales_facts 
JOIN time ON time.time_key = sales_facts.time_key
GROUP BY time.month
ORDER BY profit desc;'''
    cur.execute(sql1)
    for i in cur.fetchall():
        print(i)
    # close the communication with the PostgreSQL
    cur.close()
except (Exception, psycopg2.DatabaseError) as error:
    print(f"Error: {error}")
finally:
    if conn is not None:
        conn.close()
        print('Database connection closed.')


Connecting to the PostgreSQL database...
Question 1: How many sales did the company have?
(550720,)
Question 2: How many products did we sale in a store last month?
(5336,)
Question 3: Which one of our stores have the highest amount of sells?
(3, 43397)
Question 4: Which products are the most lucrative?
('Buffalo Jerky', 11022)
('Chicken Dinner', 7822)
('Turkey Dinner', 7265)
('Beef Stew', 6980)
('Lasagna', 6839)
('Paper Towels', 6617)
('Dry Tissues', 6404)
('Wet Wipes', 6401)
('Clear Refresher', 3374)
('Athletic Drink', 3357)
('Fizzy Classic', 3336)
('Lots of Nuts', 3322)
('Fizzy Light', 3302)
('Extra Nougat', 3279)
('Strong Cola', 3261)
('Sweet Tooth', 3253)
('Power Chips', 2899)
('Onion Slices', 2754)
('Salty Corn', 2595)
('Dried Grits', 2548)
Question 5 : What was the most lucrative year?
(1994, 54350)
(1995, 42280)
Question 5: What was the most lucrative day of the week?
('Saturday', 15185)
('Monday', 14325)
('Friday', 13681)
('Wednesday', 13528)
('Tuesday', 13513)
('Thursday', 13